In [2]:
import bibtexparser as bp
import pandas as pd
import numpy as np
import yaml as yml

# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dirConfig = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos'
configFileName = 'config_analisa_bibtex.yml'

with open(dirConfig + '\\' + configFileName) as f:
    configFile = yml.load(f, Loader=yml.loader.SafeLoader)


In [3]:
# ANALISA BIBTEX
listaArquivos = ( \
	configFile['FILE_ACM'], \
	configFile['FILE_IEE'], \
	configFile['FILE_SD' ]
	)

formatFile = configFile['FORMATO']

dirInput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\input'
dirOutput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\output'

mergedFilesCSV = 'ALL_ARTICLES.csv'
mergedFilesJSON = 'ALL_ARTICLES.json'
mergedFilesYAML = 'ALL_ARTICLES.yml'

def bibtexToDict(arqv):
	arquivo = arqv
	with open(dirInput + '\\' + arquivo, encoding='utf8') as f:
		bib_database = bp.load(f)		
	return bib_database.entries_dict.values() # retorna lista de dicionarios, verificar extend
	
# UNION DATAFRAMES AND YML
listaDf = []
listaYml =[]
for arqv in listaArquivos:
	sourceArticles = bibtexToDict(arqv)
	listaDf.append(pd.DataFrame(sourceArticles))
	listaYml.append(yml.dump(list(sourceArticles)))

unionDf = pd.concat(listaDf)
unionYml = ''.join(listaYml)

In [4]:
# EXPORT
if formatFile == 'CSV':
	unionDf.to_csv(dirOutput + '\\' + mergedFilesCSV, index=False, encoding='utf-8')
elif formatFile == 'JSON':
	unionDf.to_json(dirOutput + '\\' + mergedFilesJSON, orient = 'records')
elif formatFile == 'YAML':
	with open(dirOutput + '\\' + mergedFilesYAML, mode='w',encoding='utf8') as f:
		f.write(unionYml)
else:
	print('Formato não disponivel.')


# Article Impact

In [74]:
import pandas as pd
import numpy as np


dir_iput_aula3 = r'D:\Impacta\Python for Data Engineer\Aula 3\Input'
dir_output_aula3 = r'D:\Impacta\Python for Data Engineer\Aula 3\Output'

file_scimagojr = 'scimagojr 2020.csv'
file_jcs = 'jcs_2020.csv'
file_article_impact = 'ALL_Article_Impact.csv'

df_scimagojr = pd.read_csv(dir_iput_aula3 + '\\' + file_scimagojr, delimiter=';', quotechar='"', header=0)
df_jcs = pd.read_csv(dir_iput_aula3 + '\\' + file_jcs, delimiter=';', quotechar='"', header=0)

# TRATAMENTO
# Remove colunas vazias no arquivo
df_scimagojr.drop(df_scimagojr.columns[df_scimagojr.columns.str.contains('Unnamed:') == True], axis=1, inplace=True)
df_jcs.drop(df_jcs.columns[df_jcs.columns.str.contains('Unnamed:') == True], axis=1, inplace=True)


df_jcs.rename(columns={'Full Journal Title':'Title'}, inplace=True)

# 'Journal Impact Factor' e 'SJR' são  métricas diferentes
# df_jcs.rename(columns={'Journal Impact Factor':'SJR'}, inplace=True)
# df_scimagojr.rename(columns={'Total Cites (3years)':'Total Cites'}, inplace=True)

df_scimagojr['Title'] = df_scimagojr['Title'].str.upper()
df_jcs['Title'] = df_jcs['Title'].str.upper()

# TODAS AS LINHAS DO df_jcs QUE NAO EXISTEM NO df_scimagojr
# df_jcs_left_df_scimagojr = df_jcs.merge(df_scimagojr, how='left', on='Title', suffixes=['_jcs','_scimagojr'])
# df_jcs_exclusive = df_jcs_left_df_scimagojr[df_jcs_left_df_scimagojr['Rank_scimagojr'].isnull()] 
# df_jcs_exclusive.filter(df_jcs.columns)

# UNION
# df_scimagojr_jcs = pd.concat([df_scimagojr, df_jcs_exclusive.filter(df_jcs.columns)])
df_scimagojr_jcs = pd.concat((df_scimagojr,df_jcs))

# TRATAMENTO UNION
dict_treat_data = {'': np.nan, None: np.nan, 'Not Available': np.nan}
df_scimagojr_jcs.replace(to_replace=dict_treat_data, inplace=True)
df_scimagojr_jcs['Rank'] = np.arange(0,len(df_scimagojr_jcs))
df_scimagojr_jcs.drop_duplicates() # Deduplicação dos dados

# EXPORT
df_scimagojr_jcs.to_csv(dir_output_aula3 + '\\' + file_article_impact, index=False, encoding='utf-8')
df_scimagojr_jcs.shape




c:\Users\Saulo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


((45962, 23), (32952, 20), (13010, 5))

# Article Impact and Bibtex

In [125]:
# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dir_config_aula3 = r'D:\Impacta\Python for Data Engineer\Aula 3\config'
config_file_name = 'config_article_impact.yml'

with open(dir_config_aula3 + '\\' + config_file_name) as f:
    config_file = yml.load(f, Loader=yml.loader.SafeLoader)

filter_list = ( config_file['TITLE'],\
	config_file['KEYWORDS'],\
	config_file['YEAR'],\
	config_file['TYPE_PUBLICATION'],\
	config_file['DOI'],\
	config_file['JCR_VALUE'],\
	config_file['SCIMAGO_VALUE'],\
	config_file['FORMATO']
	)
print(filter_list)



('OVERVIEW', 'characteristics', 'D', 'E', 'F', 'G', 'H', 'CSV')


In [126]:
# TRATAMENTO
df_bibtex = unionDf
df_bibtex.rename(columns ={'title':'Title'}, inplace=True)
df_bibtex['Title'] = df_bibtex['Title'].str.upper()
dict_treat_data = {'': np.nan, None: np.nan, 'Not Available': np.nan}
df_bibtex.replace(to_replace=dict_treat_data, inplace=True)

# JOIN
df_bibtex_impact = df_bibtex.merge(df_scimagojr_jcs, how='left', on='Title', suffixes=['_1','_2'])

,Title,keywords
0,BIG DATA QUALITY METRICS FOR SENTIMENT ANALYSI...,"Big data, Big data quality metrics, Sentiment ..."
1,TOWARDS A DATA QUALITY ASSESSMENT IN BIG DATA,"Big Data, Data Quality, Quality Models, Data Q..."
2,DEFINING BIG DATA,"Big Data characteristics, Big Data, Data Quali..."
3,FROM BIG DATA TO SMART DATA: A DATA QUALITY PE...,"Data Quality, Smart Data, Big Data"
4,BIG DATA: A RESEARCH AGENDA,"big data posting, big data, OLAP over big data..."
...,...,...
253,BIG DATA ARCHITECTURE FOR CONNECTED VEHICLES: ...,"Connected vehicles, V2I communication, Automot..."
254,SEMANTIC-AWARE DATA QUALITY ASSESSMENT FOR IMA...,"Semantic-aware, Quality assessment, Image big ..."
255,BIG DATA ANALYTICS FOR CLINICAL DECISION-MAKIN...,"Analytics, Clinical decision-making, Big data,..."
256,"NUMERICAL, SECONDARY BIG DATA QUALITY ISSUES, ...","Data quality, Big data, Secondary data, Numeri..."


### FILTROS

In [141]:
# df_bibtex_impact[df_bibtex_impact['SJR'].notnull() == True][['Title','SJR','Journal Impact Factor']]

df_bibtex_impact.loc[\
	(df_bibtex_impact.Title.str.contains("{}".format(config_file['TITLE']), case=False)) |
	(df_bibtex_impact.keywords.str.contains("{}".format(config_file['KEYWORDS']), case=False))
	][['Title','keywords']]



,Title,keywords
2,DEFINING BIG DATA,"Big Data characteristics, Big Data, Data Quali..."
9,A COMPREHENSIVE OVERVIEW OF BIG DATA TECHNOLOG...,"HDFS, Apache Hadoop, Big Data Technology, MapR..."
10,A COMPREHENSIVE OVERVIEW OF BIG DATA TECHNOLOG...,"HDFS, Apache Hadoop, Big Data Technology, MapR..."
11,A COMPREHENSIVE OVERVIEW OF BIG DATA TECHNOLOG...,"HDFS, Apache Hadoop, Big Data Technology, MapR..."
12,A COMPREHENSIVE OVERVIEW OF BIG DATA TECHNOLOG...,"HDFS, Apache Hadoop, Big Data Technology, MapR..."
63,"THE 51 V'S OF BIG DATA: SURVEY, TECHNOLOGIES, ...","data storage, data characteristics, data gener..."
74,RIGOROUS MEASUREMENT MODEL FOR VALIDITY OF BIG...,"Representational Theory of Measurement,, Valid..."
110,AN OVERVIEW OF TECHNIQUES FOR CONFIRMING BIG D...,"Big Data, Method for confirming information pr..."
113,INTEGRATING DATA QUALITY REQUIREMENTS TO CITIZ...,"Data quality, Data Quality Characteristics, Da..."
145,OVERVIEW OF DATA QUALITY CHALLENGES IN THE CON...,Big data;Context;Quality management;Frequency ...


### Gerando massa de dados

In [76]:
# Subistitui df_scimagojr_jcs com 30 tilulos aleatorios do bibtex

np.random.seed(0)
title_number = 30
title_sample = list(df_bibtex.Title.iloc[np.random.randint(0,len(df_bibtex),title_number)])

for indice, title in list(zip(np.arange(0,title_number),title_sample)):
	df_scimagojr_jcs.Title[indice] = title
	print(title)


A HYBRID APPROACH TO DATA PRE-PROCESSING METHODS
THE DESIGN OF COURSE ARCHITECTURE FOR BIG DATA
EVALUATION OF DATA QUALITY OF MULTISITE ELECTRONIC HEALTH RECORD DATA FOR SECONDARY ANALYSIS
A PRACTICAL APPROACH TO SCALABLE BIG DATA COMPUTING FOR THE PERSONALIZATION OF SERVICES AT SAMSUNG
SCALING OUT BIG DATA MISSING VALUE IMPUTATIONS: PYTHIA VS. GODZILLA
QUALITY PROFILE-BASED CLOUD SERVICE SELECTION FOR FULFILLING BIG DATA PROCESSING REQUIREMENTS
DATA QUALITY ASSESSMENT FOR ON-LINE MONITORING AND MEASURING SYSTEM OF POWER QUALITY BASED ON BIG DATA AND DATA PROVENANCE THEORY
A COMPREHENSIVE OVERVIEW OF BIG DATA TECHNOLOGIES: A SURVEY
BENEFITING FROM BIG DATA IN NATURAL PRODUCTS: IMPORTANCE OF PRESERVING FOUNDATIONAL SKILLS AND PRIORITIZING DATA QUALITY
A DATA-BASED METHOD FOR INDUSTRIAL BIG DATA PROJECT PRIORITIZATION
CO-DESIGNING LEARNING MATERIALS TO EMPOWER LAYPERSONS TO BETTER UNDERSTAND BIG DATA AND BIG DATA METHODS
CONSTRUCTION AND APPLICATION OF BIG DATA ANALYSIS PLATFORM FOR ENTE

C:\Users\Saulo\AppData\Local\Temp/ipykernel_25024/2373012413.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scimagojr_jcs.Title[indice] = title
C:\Users\Saulo\AppData\Local\Temp/ipykernel_25024/2373012413.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scimagojr_jcs.Title[indice] = title
C:\Users\Saulo\AppData\Local\Temp/ipykernel_25024/2373012413.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scimagojr_jcs.Title